# IBM Cloud Hyper Protect Secure Build Server and BYOI lab

## This lab is under construction and will probably not run correctly for you as long as this warning message is here!  

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers offering:

1. Secure Build Server
2. Bring Your Own Image, or BYOI

The initial introduction of IBM Cloud Hyper Protect Virtual Servers (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you acquired.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally-signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally-signed by you or some other person or organization that you (hopefully!) trust. In the case of images digitally-signed by you, the _Secure Build Server_ feature lets you build, and digitally sign, a Docker image within an HPVS instance called a _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy your own secure build server in an HPVS instance
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample application contained within the image.

In this lab you will be using an open-source application called _Juypter Notebook_ to run the lab. This application allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal.

## Open a terminal within Jupyter

You should have two tabs open in your browser- this tab, and the Juypter home page from which your instructors guided you to this tab.

Go back to the Jupyter home page tab, and in the upper right, click the *New* button and then choose *Terminal* to start up a terminal.

From that terminal, enter the command `ibmcloud login --sso` and follow the instructions to log in to your IBM Cloud account.

You will first be given a *Y/n* prompt, reply *Y*.

At that point, the login command tries to open a browser tab, but for our lab, you are running it within a Kubernetes pod that cannot open the browser tab, so it will instruct you to copy the URL into another browser tab- go ahead an do that, and follow any necessary login prompts, and you should eventually be given a page with a login token.  Click on that token to put in your clipboard, and then, paste that into the command prompt.  When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-south
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, continue in this notebook.

You can see from the above output that no "resource group" is targeted when you log in. Run this command in order to target the _default_ resource group. "Resource groups" are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the "default" resource group which exists already.

The text you are reading now is in a type of Juypter notebook cell called "markdown".  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login --sso` above, that are run in the terminal) will be in a second type of Jupyter notebook cell called "code".  At the top of this page you can see a dropdown box that will show you whether or not you are in a _markdown_ cell or a _code_ cell.

Click on the cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top.  Then click the _Run_ button above, and that command will be run, and its output will be shown.

This will target your _default_ resource group.

In [122]:
! ibmcloud target -g default

Targeted resource group default


                      
API endpoint:      https://cloud.ibm.com   
Region:            us-south   
User:              silliman@us.ibm.com   
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902   
Resource group:    default   
CF API endpoint:      
Org:                  
Space:                


## Create an IBM Cloud IAM API Key for this lab

Run the command in the _code_ cell below in order to create an API key for your user ID. This will be used in the lab for you to store your custom-built Docker image into the IBM Cloud Container registry:

*Note:* If you are an experienced IBM Cloud user and already have some API keys, create a new one anyway, for usage just for this lab, and delete it when the lab is over. Do not use it for other purposes. If, by chance, you already have an API key named _myapikey_, choose a different name for this lab.  You can do that by double-clicking on _myapikey_ in the code cell below to highlight it and then change its name before you run the code cell.

In [ ]:
! ibmcloud iam api-key-create myapikey -d "API key for SBS tutorial"

### save the value of the API key

The output of the above command will contain an _API Key_ field.  Copy this and save it someplace safe for the duration of this lab.  (The output should remain within the notebook but save it elsewhere as well in case the output is accidentally cleared. There is no way to retrieve this value later.  The _API Key_ field should be treated as a password, as that is essentially what it is.

## Set your IBM Cloud Container Registry region to us-south

Run the code cell below in order to set your IBM Cloud Container Registry region to us-south

In [123]:
! ibmcloud cr region-set us-south

The region is set to 'us-south', the registry is 'us.icr.io'.

OK


## Create a container registry namespace

The code cell below contains a command to add a namespace to the IBM Cloud Container Registry that will be owned by you. You can try to run the command as shown, but it will definitely fail- go ahead and try it, it won't hurt anything.  It will tell you that the _secureimages_ namespace already exists.  Change _secureimages_ to something unique by adding your initials and a number or some other unique string to the namespace.  Keep trying until you get a message that indicates success.

*Note:* If you have used IBM Cloud Container Registry previously, you may already have a namespace. Feel free to either use that or create a new one for this lab.

*Jupyter Notebook tip:* If you click the *Run* button above after you've highlighted the code cell below (or any cell for that matter) it will attempt to run the code and then move to the next cell.  If you need to retry because the name you chose was already taken, just click in the cell again and try again.  Alternatively, you can use the menu and choose *Cell->Run Cell* which will run the command but then the cursor will stay in that same cell. (The *Run* button is the same as if you had chosen *Cell->Run Cells and Select Below* from the menu).

In [125]:
! ibmcloud cr namespace-add secureimages

Adding namespace 'bjs-secureimages' in resource group 'default' for account Barry Silliman's Account in registry us.icr.io...

The requested namespace is already owned by your account.

OK


Change to the HOME directory and list it:

In [ ]:
! cd ${HOME} && pwd

Download the Secure Build Command Line Interface (CLI):

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

Change into the _secure-build-cli_ directory created by the above _git clone_ command:

In [ ]:
! cd secure-build-cli && pwd && ls

In [ ]:
! cd secure-build-cli && pip3 install -r requirements.txt

In [ ]:
%%bash
cd secure-build-cli
cat <<EOF > sbs-config.json
{
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:IBM/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "bjs-secureimages/secure-bitcoin-wallet2",
    "DOCKER_USER": "iamapikey",
    "DOCKER_PASSWORD": "<redacted",
    "DOCKER_RO_USER": "iamapikey",
    "DOCKER_RO_PASSWORD": "<redacted>",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "DOCKER_BASE_SERVER": "us.icr.io",
    "DOCKER_PUSH_SERVER": "us.icr.io",
    "DOCKER_CONTENT_TRUST_PUSH_SERVER": "https://us.icr.io:4443",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY", "INSTANCE_ID", "IAM_ENDPOINT"],
    "ARG": {
      "NO_GRPC_BUILD": "1"
    }
  }
EOF
cat sbs-config.json


Create the client certificate and client CA by running the following command:

In [ ]:
! cd secure-build-cli && ./build.py create-client-cert --env sbs-config.json

Run the following command to display the client certificate and CA in base64 encoding:

In [ ]:
! cd secure-build-cli &&  ./build.py instance-env --env sbs-config.json

Run the following to command to show that you do not currently have any HPVS instances:

In [ ]:
! ibmcloud hpvs instances

In [ ]:
%%bash
cd secure-build-cli
cat <<EOF > secure_build.asc
 -----BEGIN PGP MESSAGE-----

 hQIMA4AFxsw8YZ3rARAAnR1PB+ij0LNdueQIxtzIVnJ2ogQdh7+akYLC/V85u8Xh
 NKZmxsjwzVsCISZOSwvn/BbylonBPXG+c99gI60ChA+vjpCmXqU7dASWTjUyBfYZ
 piu6ktHRpQrsAPPZl2o+17yjFYnlRpG2FkVTfEMMAdGbC4j+g345RgF3trBgsMdF
 rvaJa2mbsekau9I076hNHKgm02ar3VHET8Y1sMGmij87cgmvKipqoyF/TIfv8C0c
 fK2CX4ty8DRMINPSojGiDTR15JBd+QwdeW9cO1fKEczKMiXiNcDawo8X0r0ICHKs
 +QssrlOwn0XG7QzQhW6DOdAjWmu5QnlCAs5+yJ47vMfLL5g+84S2Bm2Lrn6fbeVc
 cBcBZr79Bbp4uDsF8OMVfrEl2Kv3fQnTbnA3bL7k1W/ABU5ytPYnBZDo/h45xq8z
 Cpw3lVNnMXqN2+i4Dc3dF+qtSMOo3nlmEcMxfhUDooWAyXX4/cysjU9P1TCs4OsA
 inov8GM7V2nYXFTKo9rKfS7UMiqsLAKwpw6YvP2tZUoNk/G8Wyb4cjB052R28MSG
 H8VBbIouKyJhDuSJk4J2pwWCwQYFcE5CTGzIStWZpu5CdYr9TMlVNZJ1jrwp0a+o
 wlFA3WHGYczSWQa0smc8U7L/jcOVAtV0u2Q+5g3AwJUy/ogZ7WfynJ+t7H3a0WjS
 7AGCRvaiPyVroBJuaGOwFHAXE19u5/I4xHgYpDaaQlbcX2m3MVdi8N96UHnTXKvH
 QLPi7IeciT2/56eYULzynb8PuFyQbmS8WSeSJmjUB0wZB12Ys4xUAUObuHwzLgk0
 2zPRJjRM7NrDw824yJGmSDr+I9E30we8TMJX4ZGP2a2q8RrXlxLgUqgMU8QKe+wx
 p96HmaNC3L4Vahhr7/U2OQ638r/ow++xOvbhVn3X8uFmA5I9PW92AdpqtaRuKTtw
 a1+63K32ctcWaEH7DUGfBmWaqpD/QZbE/SFhMT+sHEuLXgOP/dbTdWSGtW4upkGp
 bIwrcU30gZ05J1O8s2kRc+K2m4Ty44AILT0ddhGCGJicJDLAl0JNryFP4Eazs/Q8
 WawQN6sl6Hr7f2QDjJcaxUtZ3uO7/r46AjRap8j+UP3GJEoDf2Ry1/lVO1xGTEWl
 KNo4db5Xkuxq1xNJPCigpv206eH0Fuk8G3S5EK+h7PUmZ1aQgE8GzZJs21FqI+gY
 WGqXqp98rLYFX4IjEm7be8kTWlB0QZCtfA1cqtWNRy0chrak+XYbgPPptai26Eox
 hIAInWEkIzLWC/4t7WIgIOOWwKyMdeNHj0iAlihGToqtmqtcE5pj+CyRqzA7uHUb
 w1H4+q0w92Tcf6d7+xKrZ1OuJZ+5v7j/S1ZhLCMWC0yQzEZCCK4pc8D3I6ZoMcRK
 nlgJXKznjI27A6ZwXi8yBJQu+TvJIwIpP2nTF9LK76bktsQYAAS/zcxSBlCSMkeu
 gqeE82sf32j0JvAdplbHD0uq9fQeJRJQ2nMae7qYwIy6d2XMZMWCKsKxKm9BB8bU
 qpmtGIGjHyTZTCTh/BVsaPL++jCKhLq4W/8J5C/58hhfR4gH/V6sDmmAQb85wotT
 tGnZzVDDqL1zk1Qou7+6tndNdOTWVmydp70xkJE393CngLiuzDCoZuFFC8F3XZXW
 Qx+2+upbmhoqg9QwnMZPjemb1sesMHPUJn+wRlpSbiYiaYA9qxyozuQBcq2C8zP7
 Y8Kpbsajve4dKvXz4p9S1jsRzuE1wQ0/3WjUawYQQiv08v9yurhUB2yWilfv0ypX
 1TrISFaCkHBhDK10m/2VzGDR9lr5hPcHL8KuM3Mi5XqQagvmPrdLC4UR0EP7NveX
 zkg4GkBgrQpV9Sxq9x6zndSymdCpmYVUnBRZtS94qAhfpr3riMkL1EUWvzlVp6um
 MbMBKjWhtBpvLNTeFR2qtE5FjFyeY0k08MT6W5MvtJ4tljOX65kdXDeBa+HZD+GI
 Ae3UMTrT1WR0hOcOa6nGkpyidFGyL1wZBF4AgbCsh7jQ/tiYm1oQh8lRFBvm4R9S
 dJpB7rX92QCEvcfROlXBUCGmbYxl4L8SFdqdlS7HjGLi5YYI9JeT4FylAd2r0Ehm
 HSuuG0Xlvedn34WDTrYOMoLl28Lha4tHl8WKaY4XqnNJ9JNqEvqRdVs/f5LcY6ja
 PTo4M8x51ZqZKE8WXtJeHQF0kA8I5i65TA/N/56tnCidVciM6oornTocsxMdMsW4
 y2NvsgS+vkMh+AhkaHR+rTu+FsbRwlWdd16NtLSNS4Q6H8n6+K8nzyB7q9D0f2uO
 1EODUQrM+08LRKL1tQJQSyhMcj13X8p9BqEYmAaGU8TrNRHx1+Yg+5iyTrAOSPqQ
 7oQTCojdrkE0MEQVYTvakZJs7yFYMpuO8Nq97DJg5++ntutUU0dMt0LNqfN/3xkY
 GdUodrb/lKIJSZAquFtm0zwjKW3d7qLWA9WWblLNz5yvJHSm9LSoEAe4gYnNfv1v
 VEHwjp5u0aWWWFE8Vwh6c5SeOZNbjfR9xwxdQJ3SJS59RHuWsv/A1CNR4P77Ow86
 Z4SfMJDxh7QQJ5wQnyIJdL0ESgPqDcK0yiWPzuQTuBweadlTNpOKApr8E29lfmX4
 jkRDcEDELwQEFqb2cISzo6Y+lC6aJvRnZnbaCb6zj3IGbpqFD8PXwiwTkV9yGfSY
 WkEuimQc62l2QnArATR8WHP6XBSyamS/zEoEuPPFgG38p9YHf75CB4lgXpTsNyJv
 4NxzPP4MaXpM0ESjLZVXx2TM8LA79fF2OPLap3y2I3ayodAMMalSpWhz6srRIKZ5
 b2omKyVFKlVeYzwRB+EjioKG1JrM443efyS9GEZhC75XEKuQ9mio6A2uSOg8fe4O
 h3JPTfwcXKWMmcwg3rhptmCAQNkzrNFBa3k62/mkf3fzilJHsr3Koby3G73lsgww
 8AISMEeh8nOG4k60CoqMQisG6vOG8fUaI8migwcipbbmbIcDU4+v4A9zPVO+gBDk
 c5zF1qMrvCITsofX5LBg99e59+I23htk/rOBae+ShSyetYx7BC1jM7qLSz364lnV
 PEHEeCKOCJlLQYMdOTM3dOExcb2YI3bnG7vqKShCSwvytiidVb1O95uUCP1/rOH0
 jkW20EPsRqp3qW2RqSaDwhvMpfl/cFnR9dZmkHjM4ApGhmGEUGuI9WGHUw/S6EN9
 NTR/POdrKwRKs9rKAlccHywFOMttXWbtSTHpDPWBA38YKsOMFUc61JHSYuPd1kLV
 EOImT8cymuMhAivtImipd17CY4DND6XhEpTFtm8Opb5a9P+fK3paKlqPlT23n6BM
 83FIPPqQT4y5buD9hCy+bf3CkAcBVRAUfOyPaVWvXR+Ihcl6UhTXSOyzXI/2JNmX
 lfTOtpVgJImrr0/jn/5yrNxJNoeVsAVv4gP7YarWBALcnn3QU2FLDKCAVG5OtSCr
 XkQ6hy5Sr4CdPbHU0SDRtjfiS1vu44Gdi9Uck/wMi9D6LFIq1k0OG2ozXPSgiN/S
 cvHTlF9oD/E2DyjABM4IbVL+KutEx27/jBN9sZJlPGhVibBlQfSnTkFqk2sC80rB
 fF9i0QxiqmT+bZ4iK8AHsEcbc2bv4ia3qxUMy6wPUmQx0MFqtmf+B9MauJH88qnQ
 LjfrmTm0KStILsux3cPTCIXzQw4at9E7Dihxoti973Buh9HX2lxLeHCuiuxmUmtO
 XAwshmHDC7W/GLq5uN3k4gq9pa5N5RA+W0rM4iUiRq//GkFUwDo9xFGD+lKK4D6H
 YqZ/cQwnLNOKtwyCIyL2fBgj/7TtaIy5PhQJDvPvcpKBSUzUGzcqnCWgy9v/Ma4z
 pShhe8G7UDPTiYWI5iYM5SuUanskHnndU2hCdOoD/ZMkoyKvKbQ2/AyinXuA02v9
 jyrFRbbHFRV/Me/x+DUn7dJpXRsfC9lmtBV1Ev6K9nJGJU4tcYkAwxkNrFEW9q9f
 q8n5/jkDUHvQGgEXvKBJCRPJ+HC+kninVv4+eSeWAWGqRFQaedCqPVdxXprqfSvI
 LbxhAMW6XspZPREqw8HLxgZD9siKXEYf26X0FLjX3INoolHedQvWD1fn2DgcZAsJ
 R2714XbSnjErB4/wkw/4bQOzvu2fSBEI7Cnf7WbZzv8gft9r8sNJ7xc1F7WZtbkS
 CrMmTojBA7av5aS8csHiqpxpEaCgFARtiapobhnqElEMRAfwXej9fXVHmlDgeamB
 Zt8pIOYrjNjj3eDjcA4hI5iuRh+/UG3djIG7sCx2COFvbRiAMfXw+WJaMyhMWgzi
 b8R4LF+a2mc6TOCZFaP2ByfOw5T/uz3pL0pw5+fn44+W5fND2UQ1fOS7KDcafQMd
 8ca+pzpm1EBLglbKVdDCu3Zj3JbwdibJz3447v6G5dPPY4JZIoH8FLLXbjQzSWil
 qYCPcjo26QgmuxyJdCHVpWSMUFzUEXgXtCmaFDjvp5uGReL09WlBSGa+GSe0plDW
 dCtTs2VCGqLA45yorjZXKDPSFjS3iQmpFQDkldlAoo4w/d+PKIyy52UiBawXuX96
 sBrxes+jTT8ho2azOAy0BfY+TJwg94YUhrBwksmzUHUaQA7gUcROIwpvXDQqLLLi
 YPTHaqhplKPRbRHGYOy4PHougSEf2oUsieDxG6XfVTjh3zXCgBST3/zlnFSLXxFD
 l8TPdu0UNm1+n9amgfS7c1a0xu/P1WO8cjZOBSZqeN2ehkn6mLSR6wqj+eVZxe4G
 x0cDXTC6+13kBJG81RVqfxUWBOOwAg56rFKLhkMdbhhrPWtcwYU+dEagkz9oG/YA
 gQ0AAz8csvTSS3LR723k+PePv+7EKhnGKgNDKJUVSfaz9bf9jXXqF6w0ytimvdbU
 lStm/jbgkRKhrqQW6DfLPd9PviqwsaiqMhmf/HHhCDv1NAshVuV8G23FUYCoYJz4
 KlDyXn02OqSsqx4JhRBRQyo2h1+CVSaEuaK4QyAQfWKhwYxeqrkI9oy5pq3uKJGJ
 kDdDbJfFRDk08FQ5NuZ70zCQGiYM86YYRSLz+5dXYUzwZ/vsRYrDx1XCNqMnz3Y5
 vUyYw7EtKP4XTlWK6ZfIJ+NhO4wifHVxQGmXbvjVBD2VEb1nJSZHg9ojzLqrdcNx
 ckTsfV/YzIkUO0ENmRda2neEQFTGEV6NtSGIC77tHmygkLJ5IqKBQOvjZ+mhgehp
 jwQT+MmRc9iItYjUnT/pZgz85WNOS526KP/VKIF5abReAg6+xJ3GVYxpjO6qMjWi
 SXWoB3PAIyhCxrgVc2tHKcGUT9QmnUCahyG//QqzPmROxauv0V8R8DdLhaB+UlIZ
 Dxv4HQTZwdcqWntqILJHpw8YH7NLPT2VaqAUtsCpzkMLQS4ZQH+kQv6LlcmgSWOs
 1SDgII8h2oNrGG4/pjMv4N1s43a4E5AmFMFDM4Cv+P8I0MzwhagOFS9d+s3144XF
 1jdshZ9X/bemjnv2nK+84bued6t28+te8BQdGIKYvajjlpmcB/xV6hrbkIrStxty
 tuThisoMY98yv3RUBU63J4hUOINNzllHWDubRvzBD3eZeEeaW5tEeGL9/aUrNmt3
 3TgYlmIvm1wBM6kzZ8JH+RS/yf3TXGimNLwTtiBEVHnbWIbqLhvIsCpKB37n9UNo
 c8dLxNRPWt9G+XDRvi43VB4d54K7BETfs2fDz12pI/WlDaqxBZyHlnjpo3R1410T
 NmzkjBrBENr9RRzuSNjkLvDNXxr969VTgVvILkp8cH5IewaMiwdTqWApzSkLQ5Gq
 LNqKOnDGIMx4v5RPQrhI6s9RqhFHthQsS8hM+VMuOSGoiC0aYK3s6TpN536n/yIP
 jjPMDhqBk/DWBO+N7kg/zeF8KvMi4dT3lJc0wMiYAwtqA8gHyZNYZhpg+eyjzswZ
 F7CDW0++NDsHjkrVoQbdyuGAbXg1tfo9PHagLJcI1O/n2lYSSXHlsxCxps6ih8WV
 Wbu+07JCGQ+AEfGrqSOHBF5mtj/iWAH3xQChg6atLw3uem39XUIkYnr+XC4bMmRD
 U8z97xeIFrmd7JhbxtKi5AoHlDrfRkSn2ZGD1o1ouGAwhRpJUpNjcMv55UzokyCe
 cEUSsg5ElfK2f6+5ZyVWHPsJ9FxfjDM/AUyQlViveJxRCSz2LpJ63hTa9LyTBhqW
 QZQ6775FHtW837lrmheRFoXp27ak8y4JnRX1VPIpqAzwxaVg/oX+jkDJkUce4jAV
 2dDKYz+VnjIv82Farx9wMaW3K5y5zH5QDr+x15oYmez8Sb2wxmGTwZfNP5eZagmd
 hP5NX5Q21qUk3XX1SDnXtIRIjgLIJrygfU6hgLdYTPX44Tnc8JtW6TQpEbuaFy5J
 TCqzVuxzoWzyEDK1DKZPINxSs2GsLIadV7Q4Wkk9beK6RwchMmNPQJBmQZ7uJhO0
 D0PvksiYV2bTLojuhW1IYUwe3xHroCLOzdPItH/Lq0eCSpLtXifUC1xtcPmZLCrB
 SwOU0/bwK82CVoKcHlhOkzmSP/KvE4CmYrYvuOPnv47gfZcN1EkmDEBYTTsGI4FS
 fjtwcGjEiAJTuCIySWJaJwpIdJ7wmjgXbi1kkHcTYqRTXLqhWiesF2omap/Nzcmx
 6fY43ANcIBMOeFGO5pcRfpLzX611icSeulNRQlFqb+OQtFeYRHc7/I6oYsguOvmZ
 j6JTQzQOND25+jcv1WsW3myBjryHgbm1UlTcxH4xwiczNXoHJ8va+g+SBjP+oux9
 B+KZTpBHG9YfkPFOHUMTd42lZtO8G9KloJchIPfZdNiheIIHQGd3cX1rzhNrxTK/
 1725LUSciSacXRp3wD03zMkGdXXfn/iv0shpayxfBpbABuM3vDjOLD9bA8D2jfTN
 MJe9pyb9ek6y8yctWncA9Ii2mYcC438yL+0LZ+SD1PVyOB2m5xQ0Gq3YXvJOvtIU
 a1wEfTEn8DddBJNk6jn1RAIwJzuC2Doy1O5Wskybwh6UXzbeV8zuP+7q/wupn1Ox
 yLTXN/pB+lAu9C6vYnW52nQrE9nZLO3Kwh8/yU8uUDYs0i4qeUwMCeQWjn5nXc05
 gNDz+Zb9bKCS60azbldfCF01MOixCWNVGeSb66Zz2asSKCWj7DHQ4LedTRZzoZeR
 =aq7v
 -----END PGP MESSAGE-----
EOF


In [ ]:
! cd secure-build-cli && ibmcloud hpvs instance-create SBContainer lite-s dal13 --rd-path "secure_build.asc" -i 1.3.0 -e CLIENT_CRT=<redacted> -e CLIENT_CA=<redacted>


It will take a few minutes for your instance to be built. Paste your unique instance id (shown in the output of the above command, it starts with _crn:_, into the command below and run it periodically until the instance is successfully created.

In [ ]:
! ibmcloud hpvs instance crn:v1:bluemix:public:hpvs:<redacted>

In [ ]:
! cd secure-build-cli && cat sbs-config.json

In [ ]:
! export PUBLIC_IP="<redacted>"  && echo ${PUBLIC_IP}

In [ ]:
! echo ${PUBLIC_IP}

In [ ]:
! cd secure-build-cli && sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json --noverify

Run the following command to get the server CSR:

In [ ]:
! cd secure-build-cli && ./build.py get-server-csr --env sbs-config.json --noverify

Use the following command to sign the server CSR:

In [ ]:
! cd secure-build-cli && ./build.py sign-csr --env sbs-config.json

In [ ]:
! cd secure-build-cli && ./build.py post-server-cert --env sbs-config.json --noverify

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json

In [ ]:
! cd secure-build-cli && ./build.py init --env sbs-config.json

In [ ]:
! cd secure-build-cli && ./build.py build --env sbs-config.json

Check the status of your build with this command:

In [ ]:
! cd secure-build-cli && ./build.py status --env sbs-config.json

Check the logs of your build with this command:

In [ ]:
! cd secure-build-cli && ./build.py log --log build --env sbs-config.json

Run this command to download the state image (the backup of your state):

In [ ]:
! cd secure-build-cli && ./build.py get-state-image --env sbs-config.json

download the encrypted registration file for your application container image:

In [ ]:
! cd secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-ad52e76-1

In [ ]:
! cd secure-build-cli && ibmcloud hpvs instance-create securewallet lite-s dal13 --rd-path sbs.enc -i s390x-v1-369f09c

In [ ]:
! ibmcloud hpvs instance crn:v1:bluemix:<redacted>

[Change the x.x.x.x to your public IP and then click here](https://x.x.x.x/electrum)